In [ ]:
#------Importing Packages-----#
import warnings
warnings.filterwarnings('ignore')
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import os
from scipy.io import mmread
from scipy.sparse import csr_matrix
import seaborn as sns
#Spatial data imports
import spatialdata_io
import spatialdata as sd
import spatialdata_plot
from spatialdata import get_extent
from spatialdata import bounding_box_query

In [ ]:
#-------Reading in Spatial Object: Donor 1-------#
sdata = sd.read_zarr("/projects/b1217/HHA/Spatial/Zarrs/EL_S2")
sdata

In [ ]:
#-------Plotting H&E Image: Full and Low Resolution------#
axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten()
sdata.pl.render_images('20240612EL2_full_image').pl.show(ax=axes[0], title="Full", coordinate_systems="global")
sdata.pl.render_images("20240612EL2_lowres_image").pl.show(ax=axes[1], title="Lowres", coordinate_systems="global")

In [ ]:
#-------Formatting Gene Names-------#
#creates ensembl and symbol ID columns and replaces index with ENSEMBL
sdata["square_008um"].var["ensembl_id"] = sdata["square_008um"].var["gene_ids"]
sdata["square_008um"].var['symbol'] = sdata["square_008um"].var_names
sdata["square_008um"].var.set_index('symbol', drop=True, inplace=True)
sdata["square_008um"].var_names_make_unique()

In [ ]:
#------Loading in Mapped Bulb Data------#
IFE_spatial = sc.read('/projects/b1217/HHA/IFE_Spatial_Mapping/IFE_Spatial_Annotated_1_14_26.h5ad')
IFE_spatial

In [ ]:
#------Filtering Data for Mapped Spots-----#
#Adding barcodes to metadata
sdata.tables["square_008um"].obs['barcode'] = sdata.tables["square_008um"].obs_names
print(sdata.tables["square_008um"].obs.head())
#Pulling anndata from object
adata = sdata.tables["square_008um"]
#Getting filter for cells in the upper HF/SG
mask = adata.obs["barcode"].isin(IFE_spatial.obs_names).values
#Filtering AnnData
adata_sub = adata[mask, :].copy()
adata_sub

#-----Matching Spot Order-----#
IFE_spatial = IFE_spatial[adata_sub.obs_names,:]
print(np.sum(IFE_spatial.obs_names == adata_sub.obs_names))

#------Adding metadata to subsetted anndata------#
adata_sub.obs = adata_sub.obs.join(IFE_spatial.obs, how = "left", on = "barcode", lsuffix = "L_", rsuffix = "R_")

#-----Adding new AnnData to SData Object-----#
sdata.tables["square_008um"] = adata_sub
sdata.tables["square_008um"]

#------Filtering shapes to those in mask------#
#list of shapes found in the filtered spatial object
keeplist = sdata.tables["square_008um"].obs["location_id"].unique()
#All shapes
sh = sdata.shapes['20240612EL2_square_008um']
#Filtering for shapes in anndata
mask = sh.index.isin(keeplist)
sh_sub = sh[mask].copy()
sdata.shapes['20240612EL2_square_008um'] = sh_sub

In [ ]:
#----------Checking Spatial Metadata----------#
sdata.tables["square_008um"].obs

In [ ]:
#------Plotting TACCO Predicted CellTypes-----#
#Setting Palette and Plotting Order
IFEPal = ["#2E8289FF", "#B4674EFF", "#EAAE37FF", '#388F30FF']
IFEGroups = ["Basal", "Spinous", "Early_Granular", "Late_Granular"]

#------Plotting on Full Image-------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="IFEAnnotation",
  palette = IFEPal,
    groups = IFEGroups,
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/IFE_Spatial_Mapping/Plots/IFE_TACCO_Mapping_Spatial_Grayscale.png",
            dpi = 300, pad_inches = 0.5, bbox_inches = "tight")

In [ ]:
#------Plotting TACCO Probabilities: Basal------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="Basal_Score",
  cmap = "magma",
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/IFE_Spatial_Mapping/Plots/IFE_Spatial_TACCO_Basal_Score.png",
            dpi = 300, pad_inches = 0.5, bbox_inches = "tight")

In [ ]:
#------Plotting TACCO Probabilities: Spinous------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="Spinous_Score",
  cmap = "magma",
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/IFE_Spatial_Mapping/Plots/IFE_Spatial_TACCO_Spinous_Score.png",
            dpi = 300, pad_inches = 0.5, bbox_inches = "tight")

In [ ]:
#------Plotting TACCO Probabilities: Early_Granular------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="Early_Granular_Score",
  cmap = "magma",
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/IFE_Spatial_Mapping/Plots/IFE_Spatial_TACCO_Early_Granular_Score.png",
            dpi = 300, pad_inches = 0.5, bbox_inches = "tight")

In [ ]:
#------Plotting TACCO Probabilities: Late_Granular------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="Late_Granular_Score",
  cmap = "magma",
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/IFE_Spatial_Mapping/Plots/IFE_Spatial_TACCO_Late_Granular_Score.png",
            dpi = 300, pad_inches = 0.5, bbox_inches = "tight")